In [1]:
import numpy as np

### Instantiate the prediction task

In [2]:
from jenga.tasks.income import IncomeEstimationTask

task = IncomeEstimationTask(seed=42)

### Task details

In this task, we try to predict the income level of a person (more or less than 50K dollars per years) from demographic and work-related data. It is often used as a proxy to study automated decision making for loan applications.

The original data is available in the UCI machine learning repository at https://archive.ics.uci.edu/ml/datasets/adult

In [3]:
task.train_data

,workclass,occupation,marital_status,education,hours_per_week,age
24150,Self-emp-not-inc,Prof-specialty,Married-civ-spouse,Bachelors,40,35
13947,Private,Exec-managerial,Married-civ-spouse,Bachelors,50,45
30818,Private,Adm-clerical,Divorced,Some-college,40,50
5318,Local-gov,Protective-serv,Married-civ-spouse,HS-grad,56,39
3771,Private,Priv-house-serv,Divorced,HS-grad,40,51
...,...,...,...,...,...,...
6531,Private,Adm-clerical,Widowed,7th-8th,35,31
7112,Private,Exec-managerial,Married-civ-spouse,Bachelors,60,36
11159,NaN,NaN,Married-civ-spouse,Some-college,50,33
15442,Private,Machine-op-inspct,Divorced,HS-grad,40,26


### Train the provided baseline model

Jenga allows us to easily train and evaluate a logistic regression model for this task. Have a look at https://github.com/schelterlabs/jenga/blob/master/jenga/tasks/income.py if you want to know the details.


In [4]:
model = task.fit_baseline_model(task.train_data,task.train_labels)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   14.3s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   16.7s finished


In [5]:
y_pred = model.predict_proba(task.test_data)

f"The ROC AUC score on the test data is {task.score_on_test_data(y_pred)}"

'The ROC AUC score on the test data is 0.8805623332782135'

### Let's have a look at the test data

In [6]:
task.test_data

,workclass,occupation,marital_status,education,hours_per_week,age
4616,Private,Adm-clerical,Divorced,Bachelors,40,36
4012,Private,Handlers-cleaners,Married-civ-spouse,7th-8th,40,35
21267,Private,Other-service,Married-civ-spouse,Some-college,40,33
19611,Private,Sales,Married-civ-spouse,Some-college,40,39
7033,Self-emp-not-inc,Sales,Married-civ-spouse,Some-college,60,61
...,...,...,...,...,...,...
11698,Private,Sales,Never-married,11th,20,17
11969,Private,Machine-op-inspct,Married-civ-spouse,Some-college,40,51
20581,Private,Prof-specialty,Married-civ-spouse,Bachelors,50,33
31503,Private,Prof-specialty,Married-civ-spouse,Some-college,64,34


### Corruptions

Jenga provides a set of predefined data corruptions that we can use to simulate errors in the test data. We will simulate 'implicit missing values' in a column and see how this impacts the prediction quality.

Jenga supports much more predefined corruptions, have a look at https://github.com/schelterlabs/jenga/tree/master/jenga/corruptions.


In [7]:
from jenga.corruptions.generic import MissingValues

marital_status_corruption = MissingValues(column='marital_status', fraction=0.99, na_value='BROKEN')

In [8]:
corrupted_test_data = marital_status_corruption.transform(task.test_data)
corrupted_test_data

,workclass,occupation,marital_status,education,hours_per_week,age
4616,Private,Adm-clerical,BROKEN,Bachelors,40,36
4012,Private,Handlers-cleaners,BROKEN,7th-8th,40,35
21267,Private,Other-service,BROKEN,Some-college,40,33
19611,Private,Sales,BROKEN,Some-college,40,39
7033,Self-emp-not-inc,Sales,BROKEN,Some-college,60,61
...,...,...,...,...,...,...
11698,Private,Sales,BROKEN,11th,20,17
11969,Private,Machine-op-inspct,BROKEN,Some-college,40,51
20581,Private,Prof-specialty,BROKEN,Bachelors,50,33
31503,Private,Prof-specialty,BROKEN,Some-college,64,34


In [9]:
y_pred = model.predict_proba(corrupted_test_data)

f"The ROC AUC score on the corrupted test data is {task.score_on_test_data(y_pred)}"

'The ROC AUC score on the corrupted test data is 0.8043601419218791'

### Jenga's evaluators

Jenga provides a set of evaluators which allow us to automate the evaluation of the impact of given data corruptions.

In [10]:
from jenga.evaluation.corruption_impact import CorruptionImpactEvaluator

evaluator = CorruptionImpactEvaluator(task)

In [11]:
corruptions = [
    MissingValues(column='marital_status', fraction=0.99, na_value='BROKEN'),
    MissingValues(column='age', fraction=0.05, na_value=-999),
]

### Run the evaluation of the corruptions with 10 repetitions 

In [12]:
num_repetitions = 10
results = evaluator.evaluate(model, num_repetitions, *corruptions)

0/20 (0.035998000000001085)
10/20 (0.37091600000000113)


### Investigate the impact on the predictive performance of the model

In [13]:
for validation_result in results:
    
    print(validation_result.corruption)
    print(f"""
     Score (AUC) on 
      clean data:     {validation_result.baseline_score}
      corrupted data: {np.mean(validation_result.corrupted_scores)}
     """)    
    print("\n")

MissingValues: {'column': 'marital_status', 'fraction': 0.99, 'sampling': 'MCAR', 'na_value': 'BROKEN'}

     Score (AUC) on 
      clean data:     0.8805623332782135
      corrupted data: 0.8039859566886243
     


MissingValues: {'column': 'age', 'fraction': 0.05, 'sampling': 'MCAR', 'na_value': -999}

     Score (AUC) on 
      clean data:     0.8805623332782135
      corrupted data: 0.8450253502187687
     


